In [6]:
!pip install arcgis
!pip install ipython
!pip install ipywidgets  

gis)
  Using cached https://files.pythonhosted.org/packages/b1/08/ad1ae7262c8146bee3be360cc766d0261037a90b44872b080a53aaed4e84/keyring-19.2.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9e/4b/3ab2720f1fa4b4bc924ef1932b842edf10007e4547ea8157b0b9fc78599a/pywin32_ctypes-0.2.0-py2.py3-none-any.whl
distributed 1.21.8 requires msgpack, which is not installed.
Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: 'C:\\Program Files (x86)\\Microsoft Visual Studio\\Shared\\Anaconda3_64\\Lib\\site-packages\\winkerberos.cp36-win_amd64.pyd'
Consider using the `--user` option or check the permissions.

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip

## Setup supporting packages

In [3]:
from IPython.display import display, HTML

from arcgis.gis import GIS
from arcgis.features import FeatureLayer
from arcgis.geocoding import geocode

import requests

ModuleNotFoundError: No module named 'arcgis'

## HUC12 of interest

In [4]:
#huc12 = '051302030106' # Tennessee
#huc12 = '020700100204' #DC
#huc12 = '020700100103' #DC and VA
#huc12 = '031501060703' #Alpine, al
huc12 = '071000041003' #Des Moines, Iowa

# Query ArcGIS based watershed web service

In [5]:
huc_result = None

huc_ags_sevice_url = 'https://inlandwaters.geoplatform.gov/arcgis/rest/services/NHDPlus/WatershedBoundaryDataset/MapServer/10'
huc12_conus = FeatureLayer(huc_ags_sevice_url)

huc_result = huc12_conus.query(where="HUC12='" + huc12 + "'", 
                                    out_fields='HUC12,Name,AREAACRES,AREASQKM,STATES',out_sr=4326)

if huc_result == None or huc_result.sdf.empty:
    print("\nUnable to locate HUC12 information for " + huc12)
else:    
    display(huc_result.sdf)

NameError: name 'FeatureLayer' is not defined

## Display watershed result on map

In [12]:
if huc_result == None:
    print("\nSkipping as there is no data for HUC12= " + huc12)
else:
    gis = GIS()
    map1 = gis.map()
    map1.basemap = "gray"
    map1.height = '650px'
    map1.clear_graphics()

    symbol = {
      "type": "esriSFS",
      "color": [230, 76, 0, 255],
      "outline": {
        "type": "esriSLS",
        "color": [0, 0, 0, 255],
        "width": 0.75,
        "style": "esriSLSSolid"
      },
      "style": "esriSFSSolid"
    }

    map1.draw(huc_result,symbol=symbol)
    map1.extent = huc_result.sdf.spatial.full_extent

    display(map1)
        
    huc_alpha_symbol = {
      "type": "esriSFS",
      "color": [230, 76, 0, 50],
      "outline": {
        "type": "esriSLS",
        "color": [0, 0, 0, 255],
        "width": 0.75,
        "style": "esriSLSSolid"
      },
      "style": "esriSFSSolid"
    }

MapView(layout=Layout(height='400px', width='100%'))

## Retrieve assessment units from ATTAINS HUC12 web service

In [13]:
url = 'https://attains.epa.gov/attains-public/api/huc12summary?huc=' + huc12
print("\nWeb service url = " + url)
response = requests.get(url)        # To execute get request 
#print(response.status_code)     # To print http response code  
#print(response.text)            # To print formatted JSON response 

data = response.json()

assessmentUnits = data['items'][0]['assessmentUnits']

assessmentUnitList = []
for unit in assessmentUnits:
    assessmentUnitList.append(unit['assessmentUnitId'])

assessmentUnitList = ",".join(map(lambda x: "'" + str(x) + "'",assessmentUnitList))

if assessmentUnitList == "":
    print("\nNo assessment units for HUC = " + huc12)
else:
    print("\nThe following asseessment untis where found for HUC = " + huc12 + "\n")
    print(assessmentUnitList + "\n")


Web service url = https://attains.epa.gov/attains-public/api/huc12summary?huc=071000041003

The following asseessment untis where found for HUC = 071000041003

'IA 04-UDM-6316','IA 04-UDM-1211','IA 04-UDM-1210'



## Retrieving information about the assessments from the ArcGIS Service table layer

In [15]:
query_result_table = None

if assessmentUnitList == "":
    print("\nSince there are No assessment units for HUC = " + huc12 + ", we're skipping this cell")
else:
    from arcgis.geometry import filters

    lyr_url = 'https://inlandwaters.geoplatform.gov/arcgis/rest/services/ATTAINS_Geo/ATTAINS_Assessments/MapServer/4'
    attains_table_featurelayer = FeatureLayer(lyr_url)
    query_result_table = attains_table_featurelayer.query(where="assessmentunitidentifier in (" + assessmentUnitList + ")", 
                                                     out_fields='assessmentunitidentifier, assessmentunitname, organizationid, ircategory,isassessed,isimpaired,on303dlist,hastmdl,hasotherplan')
       
    import urllib.parse
    url_str = "assessmentunitidentifier in (" + assessmentUnitList + ")"
    encoded_str = urllib.parse.quote_plus(url_str)
    print("Underlying web service call to inspect the data further is availabe @")
    full_web_service_str = "https://inlandwaters.geoplatform.gov/arcgis/rest/services/ATTAINS_Geo/ATTAINS_Assessments/MapServer/4/query?outFields=*&returnGeometry=false&where=" + encoded_str
    print (full_web_service_str)
    
    
if assessmentUnitList != "":
    if query_result_table == None or query_result_table.sdf.empty:
        print("Unable to query ArcGIS Service table layer.")
    else:
       display(query_result_table.sdf)

Underlying web service call to inspect the data further is availabe @
https://inlandwaters.geoplatform.gov/arcgis/rest/services/ATTAINS_Geo/ATTAINS_Assessments/MapServer/4/query?outFields=*&returnGeometry=false&where=assessmentunitidentifier+in+%28%27IA+04-UDM-6316%27%2C%27IA+04-UDM-1211%27%2C%27IA+04-UDM-1210%27%29


,OBJECTID,assessmentunitidentifier,assessmentunitname,hasotherplan,hastmdl,ircategory,isassessed,isimpaired,on303dlist,organizationid
0,64447,IA 04-UDM-6316,DMACC Pond,N,N,1,Y,N,N,21IOWA
1,66042,IA 04-UDM-1211,Des Moines River,N,N,5,Y,Y,Y,21IOWA
2,66043,IA 04-UDM-1210,Des Moines River,N,N,5,Y,Y,Y,21IOWA


## Retrieve linear features from ArcGIS based on assessment list

In [16]:
query_lines = None

if assessmentUnitList == "":
    print("\nSince there are No assessment units for HUC = " + huc12 + ", we're skipping this cell")
else:
    linear_lyr_url = 'https://inlandwaters.geoplatform.gov/arcgis/rest/services/ATTAINS_Geo/ATTAINS_Assessments/MapServer/1'
    attains_lines_featurelayer = FeatureLayer(linear_lyr_url)
    query_lines = attains_lines_featurelayer.query(where="assessmentunitidentifier in (" + assessmentUnitList + ")", 
                                                     out_fields='*',out_sr=4326)
    if query_lines == None or query_lines.sdf.empty:
        print("There are 0 linear GIS features available.")
    else:
        print("There are" , len(query_lines.features) , "linear features available.\n")
        for f in query_lines.features:
            print(f.attributes['assessmentunitidentifier'] + " - " + f.attributes['assessmentunitname'])

There are 2 linear features available.

IA 04-UDM-1211 - Des Moines River
IA 04-UDM-1210 - Des Moines River


## Display linear features on map

In [17]:
if query_lines == None or query_lines.sdf.empty:
        print("There are 0 linear GIS features available to map.")
else:
    gis = GIS()
    map3 = gis.map()
    map3.basemap = "gray"
    map3.height = '650px'
    map3.clear_graphics()

    symbol = {
        "color": [
            0,
            92,
            230,
            255
        ],
        "width": 2,
        "type": "esriSLS",
        "style": "esriSLSSolid"
    }

    map3.draw(huc_result,symbol=huc_alpha_symbol)
    map3.draw(query_lines,symbol=symbol)
    map3.extent = huc_result.sdf.spatial.full_extent

    display(map3)

MapView(layout=Layout(height='400px', width='100%'))

## Retrieve point features from ArcGIS based on assessment list

In [18]:
query_points = None


if assessmentUnitList == "":
    print("\nSince there are No assessment units for HUC = " + huc12 + ", we're skipping this cell")
else:
    points_lyr_url = 'https://inlandwaters.geoplatform.gov/arcgis/rest/services/ATTAINS_Geo/ATTAINS_Assessments/MapServer/0'
    attains_points_featurelayer = FeatureLayer(points_lyr_url)
    query_points = attains_points_featurelayer.query(where="assessmentunitidentifier in (" + assessmentUnitList + ")", 
                                                     out_fields='*',out_sr=4326)
    if query_points == None or query_points.sdf.empty:
        print("\nThere are 0 point GIS features available.")
    else:
        print("\nThere are" , len(query_points.features) , "point features available.\n")
        for f in query_points.features:
            print(f.attributes['assessmentunitidentifier'] + " - " + f.attributes['assessmentunitname'])


There are 1 point features available.

IA 04-UDM-6316 - DMACC Pond


## Display point features on map

In [19]:
if query_points == None or query_points.sdf.empty:
        print("\nThere are 0 point GIS features available to map.")
else:
    gis = GIS()
    map4 = gis.map()
    map4.basemap = "gray"
    map4.height = '650px'
    map4.clear_graphics()

    symbol = {
      "type": "esriSMS",
      "color": [0, 112, 255, 230],
      "angle": 0,
      "xoffset": 0,
      "yoffset": 0,
      "size": 12,
      "style": "esriSMSCircle",
      "outline": {
        "type": "esriSLS",
        "color": [0, 0, 0, 255],
        "width": 0.75,
        "style": "esriSLSSolid"
      }
    }

    map4.draw(huc_result,symbol=huc_alpha_symbol)
    map4.draw(query_points,symbol=symbol)
    map4.extent = huc_result.sdf.spatial.full_extent

    display(map4)

MapView(layout=Layout(height='400px', width='100%'))

## Retrieve area features from ArcGIS based on assessment list

In [20]:
query_areas = None


if assessmentUnitList == "":
    print("\nSince there are No assessment units for HUC = " + huc12 + ", we're skipping this cell")
else:
    areas_lyr_url = 'https://inlandwaters.geoplatform.gov/arcgis/rest/services/ATTAINS_Geo/ATTAINS_Assessments/MapServer/2'
    attains_areas_featurelayer = FeatureLayer(areas_lyr_url)
    query_areas = attains_areas_featurelayer.query(where="assessmentunitidentifier in (" + assessmentUnitList + ")", 
                                                     out_fields='*',out_sr=4326)
    if query_areas == None or query_areas.sdf.empty:
        print("\nThere are 0 area GIS features available.")
    else:
        print("\nThere are" , len(query_areas.features) , "area features available.\n")
        for f in query_areas.features:
            print(f.attributes['assessmentunitidentifier'] + " - " + f.attributes['assessmentunitname'])


There are 0 area GIS features available.


# Display area features on map

In [21]:
if query_areas == None or query_areas.sdf.empty:
        print("\nThere are 0 area GIS features available to map.")
else:
    gis = GIS()
    map5 = gis.map()
    map5.basemap = "gray"
    map5.height = '650px'
    map5.clear_graphics()

    symbol = {
          "type": "esriSFS",
          "color": [ 0,
                    92,
                    230,
                    255],
          "outline": {
            "type": "esriSLS",
            "color": [0, 0, 0, 255],
            "width": 0.75,
            "style": "esriSLSSolid"
          },
          "style": "esriSFSSolid"
    }

    map5.draw(huc_result,symbol=huc_alpha_symbol)
    map5.draw(query_areas,symbol=symbol)
    map5.extent = huc_result.sdf.spatial.full_extent

    display(map5)


There are 0 area GIS features available to map.
